In [ ]:
# STEPS test(https://github.com/CNS-OIST/STEPS_Example/blob/master/python_scripts/API_2/ip3/ip3.py)
import steps.interface

from steps.model import *
from steps.geom import *
from steps.sim import *
from steps.saving import *
from steps.rng import *

import sys
import pylab
import numpy

r = ReactionManager()

#########################
# Model setup
#########################
mdl = Model()
with mdl:
    # Species 
    Ca, IP3, R, RIP3, Ropen, RCa, R2Ca, R3Ca, R4Ca = Species.Create()
    # Surface system
    surfsys = SurfaceSystem.Create()
    # Reactions
    with surfsys:
        # IP3 binding
        (IP3.o + R.s <r[1]> RIP3.s) + Ca.o <r[2]> Ropen.s
        r[1].K = 1000e6, 25800
        r[2].K = 8000e6, 2000

        # Ca binding
        (((R.s + Ca.o <r[7]> RCa.s) + Ca.o <r[8]> R2Ca.s) + Ca.o <r[3]> R3Ca.s) + Ca.o <r[4]> R4Ca.s
        r[7].K = 8.889e6, 5
        r[8].K = 20e6, 10
        r[3].K = 40e6, 15
        r[4].K = 60e6, 20

        # Ca ions passing through open IP3R channel
        Ca.i + Ropen.s >r[1]> Ropen.s + Ca.o
        r[1].K = 2e8

#########################
# Geom setup
#########################
geom = Geometry()
with geom:
    # Create the cytosol and Endoplasmic Reticulum compartments
    cyt, ER = Compartment.Create()
    cyt.Vol = 1.6572e-19
    ER.Vol = 1.968e-20

    # ER is the 'inner' compartment, cyt is the 'outer' compartment
    memb = Patch.Create(ER, cyt, surfsys)
    memb.Area = 0.4143e-12

    print('Inner compartment to memb is', memb.innerComp.name)
    print('Outer compartment to memb is', memb.outerComp.name)

#########################
# RNG setup
#########################
rng = RNG('mt19937', 512, 7233)

#########################
# Solver setup
#########################
sim = Simulation('Wmdirect', mdl, geom, rng)
        
NITER = 100
ENDT = 0.201

rs = ResultSelector(sim)

RopenDat = rs.memb.Ropen.Count

CaConc = rs.cyt.Ca.Conc

sim.toSave(RopenDat, CaConc, dt=0.001)

#########################
# Run simulation
#########################

for i in range (0, NITER):
    sim.newRun()

    sim.cyt.Ca.Conc = 3.30657e-8
    sim.cyt.IP3.Count = 6
    sim.ER.Ca.Conc = 150e-6
    sim.ER.Ca.Clamped = True
    sim.memb.R.Count = 160

    sim.run(ENDT)

    pylab.plot(RopenDat.time[-1], RopenDat.data[-1], color = 'blue', linewidth = 0.1)

datTime = RopenDat.time[0]

res_mean = numpy.mean(RopenDat.data, 0)
res_std = numpy.std(RopenDat.data, 0)
res_std1 = res_mean + res_std
res_std2 = res_mean - res_std

pylab.plot(datTime, res_mean, color = 'black', linewidth = 2.0, label = 'mean')
pylab.plot(datTime, res_std1, color = 'gray', linewidth = 1.0, label='std')
pylab.plot(datTime, res_std2,color = 'gray', linewidth = 1.0)

pylab.xlabel('Time (sec)')
pylab.ylabel('# IP3 receptors in open state')
pylab.title('IP3 receptor model: %d iterations with Wmdirect'%NITER)
pylab.ylim(0)
pylab.legend()

pylab.show()

In [ ]:
#NEURON test (OCNC 2019 NEURON Tutorial 1)
from neuron import h
h.load_file('stdrun.hoc')
import matplotlib.pyplot as plt
soma = h.Section(name='soma')
soma.insert('pas')
asyn = h.AlphaSynapse(soma(0.5))
asyn.onset = 20.0
asyn.gmax = 1.0
asyn.tau = 0.1
asyn.e = 0.0
v_vec = h.Vector()              
t_vec = h.Vector()              
v_vec.record(soma(0.5)._ref_v)
t_vec.record(h._ref_t)
h.tstop = 40.0
h.run()
plt.figure(figsize=(8,4))
plt.plot(t_vec, v_vec)
plt.xlabel('Time (ms)')
plt.ylabel('Soma membrane potential (mV)')
plt.show()

In [ ]:
#NEST test (https://nest-simulator.readthedocs.io/en/stable/auto_examples/one_neuron.html)
import nest
import nest.voltage_trace
import matplotlib.pyplot as plt

nest.set_verbosity("M_WARNING")
nest.ResetKernel()
neuron = nest.Create("iaf_psc_alpha")
voltmeter = nest.Create("voltmeter")
neuron.I_e = 376.0
nest.Connect(voltmeter, neuron)
nest.Simulate(1000.0)
nest.voltage_trace.from_device(voltmeter)
plt.show()

In [ ]:
#Brian2 test (https://brian2.readthedocs.io/en/stable/examples/COBAHH.html)
from brian2 import *

# Parameters
area = 20000*umetre**2
Cm = (1*ufarad*cm**-2) * area
gl = (5e-5*siemens*cm**-2) * area

El = -60*mV
EK = -90*mV
ENa = 50*mV
g_na = (100*msiemens*cm**-2) * area
g_kd = (30*msiemens*cm**-2) * area
VT = -63*mV
# Time constants
taue = 5*ms
taui = 10*ms
# Reversal potentials
Ee = 0*mV
Ei = -80*mV
we = 6*nS  # excitatory synaptic weight
wi = 67*nS  # inhibitory synaptic weight

# The model
eqs = Equations('''
dv/dt = (gl*(El-v)+ge*(Ee-v)+gi*(Ei-v)-
         g_na*(m*m*m)*h*(v-ENa)-
         g_kd*(n*n*n*n)*(v-EK))/Cm : volt
dm/dt = alpha_m*(1-m)-beta_m*m : 1
dn/dt = alpha_n*(1-n)-beta_n*n : 1
dh/dt = alpha_h*(1-h)-beta_h*h : 1
dge/dt = -ge*(1./taue) : siemens
dgi/dt = -gi*(1./taui) : siemens
alpha_m = 0.32*(mV**-1)*4*mV/exprel((13*mV-v+VT)/(4*mV))/ms : Hz
beta_m = 0.28*(mV**-1)*5*mV/exprel((v-VT-40*mV)/(5*mV))/ms : Hz
alpha_h = 0.128*exp((17*mV-v+VT)/(18*mV))/ms : Hz
beta_h = 4./(1+exp((40*mV-v+VT)/(5*mV)))/ms : Hz
alpha_n = 0.032*(mV**-1)*5*mV/exprel((15*mV-v+VT)/(5*mV))/ms : Hz
beta_n = .5*exp((10*mV-v+VT)/(40*mV))/ms : Hz
''')

P = NeuronGroup(4000, model=eqs, threshold='v>-20*mV', refractory=3*ms,
                method='exponential_euler')
Pe = P[:3200]
Pi = P[3200:]
Ce = Synapses(Pe, P, on_pre='ge+=we')
Ci = Synapses(Pi, P, on_pre='gi+=wi')
Ce.connect(p=0.02)
Ci.connect(p=0.02)

# Initialization
P.v = 'El + (randn() * 5 - 5)*mV'
P.ge = '(randn() * 1.5 + 4) * 10.*nS'
P.gi = '(randn() * 12 + 20) * 10.*nS'

# Record a few traces
trace = StateMonitor(P, 'v', record=[1, 10, 100])
run(1 * second, report='text')
plot(trace.t/ms, trace[1].v/mV)
plot(trace.t/ms, trace[10].v/mV)
plot(trace.t/ms, trace[100].v/mV)
xlabel('t (ms)')
ylabel('v (mV)')
show()

In [ ]:
#pystan test()
import pystan
schools_code = """
data {
    int<lower=0> J; // number of schools
    vector[J] y; // estimated treatment effects
    vector<lower=0>[J] sigma; // s.e. of effect estimates
}
parameters {
    real mu;
    real<lower=0> tau;
    vector[J] eta;
}
transformed parameters {
    vector[J] theta;
    theta = mu + tau * eta;
}
model {
    eta ~ normal(0, 1);
    y ~ normal(theta, sigma);
}
"""

schools_dat = {'J': 8,
               'y': [28,  8, -3,  7, -1,  1, 18, 12],
               'sigma': [15, 10, 16, 11,  9, 11, 10, 18]}

sm = pystan.StanModel(model_code=schools_code)
fit = sm.sampling(data=schools_dat, iter=1000, chains=4)
la = fit.extract(permuted=True)  # return a dictionary of arrays
mu = la['mu']

## return an array of three dimensions: iterations, chains, parameters
a = fit.extract(permuted=False)
fit.plot()

In [ ]:
# gym test
import gym   

env = gym.make('CartPole-v1')  
env.reset()  
for _ in range(200):  
    env.render()  
    env.step(env.action_space.sample()) # take a random action  
env.close()  